#read csv file

In [0]:
df_csv = spark.read.format("csv").load("/FileStore/global2ksample.csv", header=True, inferSchema=True)
#to limit and see first 3 rows
df_csv.limit(3).display()
#filter a record
df_csv.where("priority_rank=='2'").display()
#display all columns
df_csv.columns
#change column type using select for only specific columns
from pyspark.sql.types import IntegerType
df_csv_new = df_csv.select(df_csv.priority_rank.cast(IntegerType()))
df_csv_new.printSchema()

#change column using with to modify in same dataframe
df_csv = df_csv.withColumn("priority_rank",df_csv["priority_rank"].cast(IntegerType()))
df_csv.printSchema()


#read Json file

In [0]:
#for multiline json output use this
df_json = spark.read.option("multiline","true").json("/FileStore/test_response.json")
df_json.limit(3).display()
#to view all schema
df_json.printSchema()

#create partition by date

In [0]:
#adding new column
from pyspark.sql.functions import *
df_csv = df_csv.withColumn("created_date",current_date())

df_csv.display()



#creating partition by year
df_csv.write.format('delta') \
            .mode('overwrite') \
            .partitionBy('created_date').saveAsTable("partition_table")

In [0]:
#verify the table
%fs ls